<a href="https://colab.research.google.com/github/Az-Ks/AirQo-Ugandan-Air-Quality-Forecast/blob/master/ANOTHERLGBMODEL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# PLEASE MAKE SURE TO RUN THIS ON KAGGLE TO GET THE SAME SCORE 

In [0]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [0]:
import os
import sys
import gc
import math
import random
import pickle
import pandas as pd
import numpy as np
import seaborn as sns
from tqdm.notebook import tqdm
import category_encoders as ce
import matplotlib.pyplot as plt

In [0]:
from sklearn.model_selection import KFold, StratifiedKFold, train_test_split
from sklearn.metrics import f1_score, confusion_matrix, classification_report
from sklearn.metrics import mean_squared_error
from sklearn.feature_selection import SelectFromModel
from sklearn.linear_model import Lasso, LassoCV
from sklearn.ensemble import RandomForestRegressor

In [0]:
import lightgbm as lgbm 
import xgboost as xgb
import catboost as cat
from catboost import CatBoostRegressor, Pool, CatBoostClassifier

In [0]:
seed = 2020
random.seed(seed)
np.random.seed(seed)

In [0]:
os.makedirs('MODELS/', exist_ok=True)
os.makedirs('/DATASET/CSV/', exist_ok=True)
os.makedirs('/DATASET/ZIP/', exist_ok=True)
os.makedirs('/DATASET/DOWNLOAD/', exist_ok=True)

In [0]:
mapper = {
    "GOOD": 0,
    "MODERATE": 1,
    "SENSITIVE": 2,
    "UNHEALTHY": 3,
    "V_UNHEALTHY": 4,
    "HAZARDOUS": 5
}

In [0]:
def categorize(target):
  if target <= 12:
    return "GOOD"
  elif target <=35:
    return "MODERATE"
  elif target <= 55:
    return "SENSITIVE"
  elif target <= 150:
    return "UNHEALTHY"
  elif target <= 250:
    return "V_UNHEALTHY"
  else:
    return "HAZARDOUS"

In [0]:
def split_into_days(df, features, days=5):
  width = 24
  for feature in features:
    for day in range(days):
      df[feature+'_day_'+str(day)] = df[feature].apply(lambda x: x[day*width:(day+1)*width])
    df[feature+'_target_reading_day'] = df[feature].apply(lambda x: x[-1])

In [0]:
# covert features  fron string to List of values 
def replace_nan(x):
    if x==" ":
        return np.nan
    else :
        return float(x)    

In [0]:
def aggregate_features(x,col_name):
    x["max_"+col_name]=x[col_name].apply(np.max)
    x["min_"+col_name]=x[col_name].apply(np.min)
    x["mean_"+col_name]=x[col_name].apply(np.mean)
    x["std_"+col_name]=x[col_name].apply(np.std)
    x["var_"+col_name]=x[col_name].apply(np.var)
    x["median_"+col_name]=x[col_name].apply(np.median)
    x["ptp_"+col_name]=x[col_name].apply(np.ptp)
    return x  

In [0]:
def remove_nan_values(x):
    strict = [e for e in x if not math.isnan(e)]
    if len(strict) == 0:
      strict = [np.nan]
    return strict

In [0]:
def metric(y,x):
    return np.sqrt(mean_squared_error(x,y))

In [0]:
def train_function(model,train,test,params,other_params,target_name,features,metric, model_name):
    folds_num=train.fold.nunique()
    validation=train[[id_name,"fold",target_name]].copy()
    validation["pred_"+target_name]=0
    sub=test[[id_name]].copy()
    feat_imps = pd.DataFrame()
    feat_imps['Features'] = features
    
    for fold in np.sort(train.fold.unique()):
        print("#"*50+" {} ".format(fold)+"#"*50)
        os.makedirs("model_save/{}/{}/{}".format(model_name,Experiment_name,str(int(fold))), exist_ok=True)
        X_train=train[train.fold!=fold]
        X_val=train[train.fold==fold]
        
        train_pred,validation_pred,test_pred,feat_imp=model(X_train,X_val,test,params,other_params)

        validation.loc[validation.fold==fold,"pred_"+target_name]=validation_pred
        sub[target_name]=test_pred/folds_num
        train_score=metric(X_train[target_name],train_pred)
        val_score=metric(X_val[target_name],validation_pred)
        feat_imps[fold] = feat_imp
        print("train score : {} validation score : {}".format(round(train_score,4),round(val_score,4)))
    
    final_validation_score=metric(validation[target_name],validation["pred_"+target_name])
    print("final validation score : {}".format(final_validation_score))
        
    return sub,validation,final_validation_score,feat_imps

def lgbm_model(X_train,X_val,X_test,params,other_params):
    dtrain = lgbm.Dataset(data=X_train[features], label=X_train[target_name], feature_name=features)
    dval = lgbm.Dataset(data=X_val[features], label=X_val[target_name], feature_name=features)

    model = lgbm.train(
        params=params,
        train_set=dtrain,
        num_boost_round=other_params["num_boost_round"],
        valid_sets=(dtrain, dval),
        early_stopping_rounds=other_params["early_stopping_rounds"],
        verbose_eval=other_params["verbose_eval"],
    )
    best_iteration = model.best_iteration
    train_pred=model.predict(X_train[features], num_iteration=best_iteration)
    validation_pred=model.predict(X_val[features], num_iteration=best_iteration)
    test_pred=model.predict(test[features], num_iteration=best_iteration)
    feat_imp = model.feature_importance(iteration=best_iteration)
        
    return train_pred,validation_pred,test_pred, feat_imp

def cat_model(X_train,X_val,X_test,params,other_params):
    dtrain = Pool(data=X_train[features], label=X_train[target_name], feature_names=features)
    dval = Pool(data=X_val[features], label=X_val[target_name], feature_names=features)

    model = CatBoostRegressor(**params)
    model.fit(dtrain,
              eval_set=[dval],
              use_best_model=True,
              verbose_eval=other_params["verbose_eval"],
    )

    best_iteration = model.best_iteration_
    train_pred = model.predict(X_train[features])
    validation_pred = model.predict(X_val[features])
    test_pred = model.predict(test[features])
    feat_imp = model.feature_importances_
        
    return train_pred,validation_pred,test_pred, feat_imp

In [0]:
cols = ['location', 'loc_altitude', 'km2', 'aspect',
       'dist_trunk', 'dist_primary', 'dist_secondary',
       'dist_tertiary', 'dist_unclassified', 'dist_residential', 'popn', 'hh',
       'hh_cook_charcoal', 'hh_cook_firewood', 'hh_burn_waste']

In [0]:
train = pd.read_csv('../input/airqo-ugandan-air-quality-forecast-challenge-zindi/Train (1).csv')
test = pd.read_csv('../input/airqo-ugandan-air-quality-forecast-challenge-zindi/Test (1).csv')
meta = pd.read_csv('../input/airqo-ugandan-air-quality-forecast-challenge-zindi/airqo_metadata.csv', usecols=cols)

In [0]:
sns.barplot(x='location', y='target', data=train)

In [0]:
features = ["temp", "precip", "rel_humidity", "wind_dir", "wind_spd", "atmos_press"]

days_features = [
'temp_day_0', 'temp_day_1', 'temp_day_2', 'temp_day_3', 'temp_day_4', 
'precip_day_0', 'precip_day_1', 'precip_day_2', 'precip_day_3','precip_day_4',
'rel_humidity_day_0','rel_humidity_day_1', 'rel_humidity_day_2', 'rel_humidity_day_3','rel_humidity_day_4',
'wind_dir_day_0', 'wind_dir_day_1', 'wind_dir_day_2', 'wind_dir_day_3','wind_dir_day_4',
'wind_spd_day_0','wind_spd_day_1', 'wind_spd_day_2', 'wind_spd_day_3', 'wind_spd_day_4',
'atmos_press_day_0', 'atmos_press_day_1','atmos_press_day_2', 'atmos_press_day_3', 'atmos_press_day_4']

In [0]:
for feature in features :
    train[feature] = train[feature].apply(lambda x: [ replace_nan(X) for X in x.replace("nan"," ").split(",")])
    test[feature] = test[feature].apply(lambda x: [ replace_nan(X)  for X in x.replace("nan"," ").split(",")])

In [0]:
datav1 = pd.concat([train, test],sort=False).reset_index(drop=True)
datav2 = datav1.copy()

In [0]:
for col_name in tqdm(features):
  split_into_days(datav1, features)

In [0]:
for col_name in tqdm(days_features):
    datav1[col_name] = datav1[col_name].apply(remove_nan_values)

for col_name in tqdm(days_features):
    datav1 = aggregate_features(datav1,col_name)

In [0]:
for col_name in tqdm(features):
    datav1[col_name] = datav1[col_name].apply(remove_nan_values)

for col_name in tqdm(features):
    datav1 = aggregate_features(datav1,col_name)

In [0]:
for feat in features:
  for i in range(len(datav2.loc[1, features[0]])):
    datav2[feat+f'_{i}'] = datav2[feat].apply(lambda x: x[i])

In [0]:
datav1.drop(features+days_features, axis=1, inplace=True)
datav2.drop(features+['target', 'ID','location'], axis=1, inplace=True)

In [0]:
data = pd.concat([datav1, datav2], axis=1)
#data = datav1.copy()

In [0]:
oe = ce.OrdinalEncoder(cols=['location'])
data['binned_location'] = oe.fit_transform(data['location'])

In [0]:
meta.fillna(-9999, inplace=True)

In [0]:
aggs = {
    'binned_location': ['count'],
    'target': ['mean', 'min', 'max', 'std', 'quantile', 'sum'],

}

In [0]:
meta_stats = data.groupby('location').agg(aggs)

In [0]:
meta_stats = meta_stats.merge(meta, how='inner', on='location')

In [0]:
meta_stats.rename(
    columns = {
        ('binned_location', 'count') : 'count',
        ('target', 'mean') : 'mean_target',            
        ('target', 'min') : 'min_target',
        ('target', 'max') : 'max_target',
        ('target', 'std') : 'std_target',
        ('target', 'quantile') : 'quantile_target',
        ('target', 'sum') : 'sum_target'
    },
    inplace=True
)

In [0]:
meta_stats['mean_pm2.5_per_km2'] = meta_stats['mean_target']/meta_stats['km2']
meta_stats['sum_pm2.5_per_km2'] = meta_stats['sum_target']/meta_stats['km2']
meta_stats['device_per_km2'] = meta_stats['count']/meta_stats['km2']

In [0]:
meta_stats['sum_target'] = meta_stats['sum_target'].apply(np.log2)
meta_stats['sum_pm2.5_per_km2'] = meta_stats['sum_pm2.5_per_km2'].apply(np.log2)

In [0]:
data = data.merge(meta_stats, how='left', on=['location'])

In [0]:
data['mean_temp_day_3']

In [0]:
hum_features =  list(data.filter(regex='rel_humidity_.*').columns)
temp_features = list( data.filter(regex='temp_.*').columns)  
precip_features =  list(data.filter(regex='precip.*').columns)
winddir_features = list( data.filter(regex='wind_dir_.*').columns)
windspead_features = list( data.filter(regex='wind_spd_.*').columns)
atm_features =  list(data.filter(regex='atmos_press_.*').columns)




hum_features= hum_features[36:]
temp_features=temp_features[36:] 
precip_features=precip_features[31:]
winddir_features=winddir_features[36:]
windspead_features=windspead_features[36:]
atm_features=atm_features[36:]





data[hum_features]= data[hum_features].apply(lambda x: x.fillna(x.mean()),axis=1)



data[temp_features]= data[temp_features].apply(lambda x: x.fillna(x.mean()),axis=1)


data[precip_features]= data[precip_features].apply(lambda x: x.fillna(float(0.0)),axis=1)



data[winddir_features]= data[winddir_features].apply(lambda x: x.fillna(x.mean()),axis=1)



data[windspead_features]= data[windspead_features].apply(lambda x: x.fillna(x.mean()),axis=1)




data[atm_features]= data[atm_features].apply(lambda x: x.fillna(x.mean()),axis=1)


In [0]:
data['relation1'] = data['wind_spd_118'] +data['wind_spd_119'] +data['wind_spd_120']
data['relation2'] = data['temp_89'] + data['temp_95'] + data['temp_48'] +data['temp_70'] + data['temp_88'] + data['temp_72']
data['relation3'] = data['rel_humidity_112'] + data['rel_humidity_113'] + data['rel_humidity_102'] + data['rel_humidity_42'] + data['rel_humidity_3'] 
data['relation4'] = data['atmos_press_103'] + data['atmos_press_7'] +data['atmos_press_10'] +data['atmos_press_109'] +data['atmos_press_116']

In [0]:
wind_dir_feature_ =  list(data.filter(regex='wind_dir.*').columns)
len([x for x in data.columns if 'wind_dir' in x]) , len(wind_dir_feature_)

In [0]:
def radian_conv(degree):
    """
    Return radian.
    """
    return  np.radians(degree) 



for col in wind_dir_feature_ :
  
  data[col] = data[col].apply(radian_conv)


In [0]:
train = data[data.target.notnull()].reset_index(drop=True)
test = data[data.target.isna()].reset_index(drop=True)

In [0]:
to_drop = ['min_precip_day_0', 'min_precip_day_1', 'min_precip_day_2',
       'min_precip_day_3', 'min_precip_day_4', 'min_precip', 
       'median_precip_day_0', 'median_precip_day_1', 'median_precip_day_2',
       'median_precip_day_3', 'median_precip_day_4', 'median_precip',
       ]

In [0]:
train.drop(labels=to_drop, axis=1, inplace=True)
test.drop(labels=to_drop, axis=1, inplace=True)

In [0]:
features = train.columns.difference(['ID', 'target', 'binned_location'])
select_features = train.columns.difference(['ID', 'target', 'location'])
target = 'target'

In [0]:
best_features = ['atmos_press_1', 'atmos_press_10', 'atmos_press_103',
       'atmos_press_104', 'atmos_press_105', 'atmos_press_109',
       'atmos_press_110', 'atmos_press_115', 'atmos_press_116',
       'atmos_press_14', 'atmos_press_19', 'atmos_press_2',
       'atmos_press_20', 'atmos_press_21', 'atmos_press_25',
       'atmos_press_26', 'atmos_press_27', 'atmos_press_28',
       'atmos_press_3', 'atmos_press_31', 'atmos_press_32',
       'atmos_press_33', 'atmos_press_37', 'atmos_press_38',
       'atmos_press_39', 'atmos_press_43', 'atmos_press_44',
       'atmos_press_50', 'atmos_press_51', 'atmos_press_52',
       'atmos_press_56', 'atmos_press_57', 'atmos_press_61',
       'atmos_press_62', 'atmos_press_67', 'atmos_press_68',
       'atmos_press_7', 'atmos_press_75', 'atmos_press_8',
       'atmos_press_80', 'atmos_press_81', 'atmos_press_86',
       'atmos_press_9', 'atmos_press_91', 'atmos_press_92',
       'atmos_press_93', 'atmos_press_98', 'atmos_press_99',
       'hh_burn_waste', 'max_atmos_press', 'max_precip',
       'max_rel_humidity', 
        'max_rel_humidity_day_2',
        'max_temp', 'max_wind_dir',
        'max_wind_dir_day_2',
        'max_wind_spd',
       'mean_atmos_press', 'mean_precip', 'mean_rel_humidity',
       
       
       'mean_target', 'mean_temp_day_2',
       'mean_wind_dir',
       'mean_wind_dir_day_2',
        'mean_wind_spd',
        'median_atmos_press',
       'median_rel_humidity', 
       'median_rel_humidity_day_2', 
       'median_temp',
       'median_temp_day_2', 
       'median_wind_dir',
        'median_wind_dir_day_2',
       
       'median_wind_spd',
       'median_wind_spd_day_2', 
        'min_atmos_press_day_2',
        'min_rel_humidity',
        'min_temp',
        'min_temp_day_2',
        'min_wind_dir',
       
       'min_wind_spd', 'min_wind_spd_day_2',
        'ptp_atmos_press',
       
     'ptp_precip', 'ptp_rel_humidity',
        'ptp_wind_dir',
        'ptp_wind_spd',
       'rel_humidity_102', 'rel_humidity_112', 'rel_humidity_113',
       'rel_humidity_114', 'rel_humidity_3', 'rel_humidity_42',
       'rel_humidity_63', 'rel_humidity_78', 'std_atmos_press',
       'std_precip', 'std_rel_humidity', 'std_wind_dir',
        'std_wind_spd', 'temp_0', 'temp_1',
       'temp_102', 'temp_113', 'temp_114', 'temp_118', 'temp_120',
       'temp_16', 'temp_17', 'temp_2', 'temp_20', 'temp_22', 'temp_23',
       'temp_24', 'temp_25', 'temp_30', 'temp_40', 'temp_41', 'temp_48',
       'temp_49', 'temp_5', 'temp_50', 'temp_54', 'temp_64', 'temp_70',
       'temp_71', 'temp_72', 'temp_78', 'temp_88', 'temp_89', 'temp_92',
       'temp_94', 'temp_95', 'temp_97', 'temp_target_reading_day',
       'var_atmos_press', 'var_precip', 'var_rel_humidity', 'var_temp',
       'var_wind_dir',  'var_wind_spd',
       'wind_spd_108', 'wind_spd_118', 'wind_spd_119', 'wind_spd_120',
       'wind_spd_29', 'wind_spd_target_reading_day' ,
                 'relation1','relation2','relation3',
                ]

In [0]:
fold = KFold(n_splits=20, random_state=seed)

In [0]:
import lightgbm as lgb
params = {
    'objective' :'regression',
    'learning_rate' : 0.1,
    'num_iterations': 1500,
    'max_bins': 150, 
    'max_depth' :7 ,
    'num_leaves' : 200,
    'feature_fraction': 0.64, 
    'bagging_fraction': 0.8, 
    'bagging_freq':1,
    'boosting_type' : 'gbdt',
    'metric': 'rmse' ,
     'min_data_in_leaf':15,
    'reg_lambda' :150
}



In [0]:
test_preds = []
score_oofs = []
feats = pd.DataFrame({'features': best_features}) #You can change 

for i, (tr, vr) in enumerate(fold.split(train)):
  X, Y = train.loc[tr, best_features], train.loc[tr, target]
  x, y = train.loc[vr, best_features], train.loc[vr, target]

  model = lgb.LGBMRegressor(**params)
  model.fit(X, Y, eval_set=[(x,y)], verbose=100, early_stopping_rounds=200)
  pred = model.predict(x)
  test_pred = model.predict(test[best_features])
  sc = metric(y, pred)
  score_oofs.append(sc)
  test_preds.append(test_pred)
  feats[f'Fold {i}'] = model.feature_importances_
  # print('RMSE : {}'.format(sc))
feats['Importances'] = feats.mean(axis=1)
print()
print('CV RMSE : {}'.format(np.mean(score_oofs, axis=0)))

In [0]:
print('CV RMSE : {}'.format(np.mean(score_oofs, axis=0)))

## ON KAGGLE CV RMSE ==22.69329469880266

In [0]:
test[target] = np.mean(test_preds, axis=0)
test[['ID', target]].to_csv('ANOTHERLGB.csv', index=False)

In [0]:
feats.head()

In [0]:
fig = plt.figure(figsize=(20,40))
sns.barplot(x='Importances', y='features', data=feats.sort_values(by='Importances', ascending=False));